# Frontend Demonstration
These are two examples of a frontend for your project, but don't limit yourself to this example. Think about what kind of interface makes the most sense for your application and branch out on Google to look for any libraries that you think look fun to learn and useful for your application.

> * **NOTE**: Your frontend will need to use more of your API endpoints than mine does, but this should help you get started if you want to use a similar method

#### To start, we need some data
Just like in Individual Assignment 4, we'll use the `requests` module to get our data from our API

In [ ]:
import requests
import pandas as pd

# Get data about SEC team's 2023 season PPGs
ppg_response = requests.get('http://127.0.0.1:5000/ppg?conference=SEC&season=2023')

if ppg_response.status_code == 200:

    # Plotly uses dataframes, so let's use Pandas to read in the json into a dataframe
    ppg_df = pd.read_json(ppg_response.text)
    print(ppg_df)
    
else:
    print(f"Something went wrong: {ppg_response.text}")

#### Plotting results with Plotly
Now let's look at a new library to make some nice plots of our Points-Per-Game API endpoint results. [Here](https://www.geeksforgeeks.org/python-plotly-tutorial/) is a good tutorial to help you get started with Plotly Express, and [here](https://plotly.com/python/) is all of the official documentation. Plotly is definitely not the only plotting library out there, so feel free to explore others, but I like Plotly because it allows the user to interact with your plots.

*To install*, run `pip install plotly` in your terminal.

In [ ]:
import requests
import pandas as pd
import plotly.express as px

# Get data about SEC team's 2023 season PPGs
ppg_response = requests.get('http://127.0.0.1:5000/ppg?conference=SEC&season=2023')

if ppg_response.status_code == 200:

    # Plotly uses dataframes, so let's use Pandas to read in the json into a dataframe
    ppg_df = pd.read_json(ppg_response.text)

    #--------------------------------------------------------------------
    # Now let's create a horizontal bar graph with Plotly
    #      https://plotly.com/python/bar-charts/ 
    fig = px.bar(ppg_df,
                 x="ppg", 
                 y="team", 
                 orientation='h',
                 hover_data=['rank','id','team','ppg'],
                 height=400,
                 color='team',
                 title='Team Points Per Game')
    
    # Show our graph
    fig.show()
    #--------------------------------------------------------------------

else:
    print(f"Something went wrong: {ppg_response.text}")

Plotly has a lot of optional parameters that you can use to make your plots look and behave in different ways. Let's use our `/teams` endpoint now to get some more information about each team like their primary color and use that to make our plot look better.

In [ ]:
import requests
import plotly.express as px
import pandas as pd

#--------------------------------------------------------------------
# Base URL of our API
base_url = 'http://127.0.0.1:5000'

# Get data about Teams
teams_response = requests.get(base_url + '/teams')
if teams_response.status_code == 200:
    teams_df = pd.read_json(teams_response.text)
else:
    print(f"Something went wrong: {teams_response.text}")
#--------------------------------------------------------------------

# Get data about SEC team's 2023 season PPGs
ppg_response = requests.get(base_url + '/ppg?conference=SEC&season=2023')

if ppg_response.status_code == 200:

    # Plotly uses dataframes, so let's use Pandas to read in the json into a dataframe
    ppg_df = pd.read_json(ppg_response.text)

    #--------------------------------------------------------------------
    # Merge Teams and PPG dataframes using the ID column 
    merged_df = ppg_df.merge(teams_df, left_on='id', right_on='id')
    merged_df.sort_values('ppg') # Make sure our dataframe is still sorted correctly after the merge
    print(merged_df)
    #--------------------------------------------------------------------

    # Now let's create a horizontal bar graph with Plotly and use the team colors to color code our bars
    fig = px.bar(merged_df,  #  <-- Updated to use merged dataframe
                 x="ppg", 
                 y="team", 
                 orientation='h',
                 hover_data=['rank','id','team','ppg','mascot'],  #  <-- Updated to add mascot
                 height=400,
                 color='team',
                 color_discrete_sequence=merged_df['color'],  #  <-- Added to color bars
                 title='Team Points Per Game')
    
    # Show the graph
    fig.show()

else:
    print(f"Something went wrong: {ppg_response.text}")

#### Now let's make it interactive using Dash
Dash is an open-source framework for building analytical applications, with no Javascript required, and it is tightly integrated with the Plotly graphing library.

To use Dash, you'll have to install it with `pip install dash` in your terminal.

Let's start out by looking at the **Bar charts in Dash** example from [here](https://plotly.com/python/bar-charts/)

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)


app.layout = html.Div([
    html.H4('Restaurant tips by day of week'),
    dcc.Dropdown(
        id="dropdown",
        options=["Fri", "Sat", "Sun"],
        value="Fri",
        clearable=False,
    ),
    dcc.Graph(id="graph"),
])


@app.callback(
    Output("graph", "figure"), 
    Input("dropdown", "value"))
def update_bar_chart(day):
    df = px.data.tips() # replace with your own data source
    mask = df["day"] == day
    fig = px.bar(df[mask], x="sex", y="total_bill", 
                 color="smoker", barmode="group")
    return fig


app.run_server(debug=True)

Now let's integrate our previous code into their example to create our own dashboard.

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

#--------------------------------------------------------------------
import requests
import pandas as pd

# Establish some information about our API url
base_url = 'http://127.0.0.1:5000'

# Get data about Teams
teams_response = requests.get(base_url + '/teams')
if teams_response.status_code == 200:
    teams_df = pd.read_json(teams_response.text)
else:
    print(f"Something went wrong: {teams_response.text}")
#--------------------------------------------------------------------

app = Dash(__name__)


app.layout = html.Div([
    html.H4('Team Points Per Game'),
    html.P('Season:'),
    dcc.Dropdown(
        id="season_dropdown",
        options=['2023'],
        value='2023',
        clearable=False,
    ),
    html.P('Conference:'),
    dcc.Dropdown(
        id="conference_dropdown",
        options=['SEC'],
        value='SEC',
        clearable=False,
    ),
    dcc.Graph(id="graph"),
])


@app.callback(
    Output("graph", "figure"), 
    Input("season_dropdown", "value"), 
    Input("conference_dropdown", "value"))
def update_bar_chart(season, conference):

    #--------------------------------------------------------------------
    # Get data about SEC team's 2023 season PPGs
    ppg_response = requests.get(base_url + f'/ppg?conference={conference}&season={season}')  #  <-- We can apply our filters here

    if ppg_response.status_code == 200:

        # Plotly uses dataframes, so let's use Pandas to read in the json into a dataframe
        ppg_df = pd.read_json(ppg_response.text)

        # Merge Teams and PPG dataframes using the ID column 
        merged_df = ppg_df.merge(teams_df, left_on='id', right_on='id')
        merged_df.sort_values('ppg') # Make sure our dataframe is still sorted correctly after the merge

        # Now let's create a horizontal bar graph with Plotly and use the team colors to color code our bars
        fig = px.bar(merged_df,
                    x="ppg", 
                    y="team", 
                    orientation='h',
                    hover_data=['rank','id','team','ppg','mascot'],
                    height=400,
                    color='team',
                    color_discrete_sequence=merged_df['color'],
                    title='Team Points Per Game')
        
        # return the graph
        return fig
    else:
        print(f"Something went wrong: {ppg_response.text}")
    #--------------------------------------------------------------------
    

app.run_server(debug=True)

Now, let's use some new API endpoints to populate our dropdowns.

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import requests
import pandas as pd

# Establish some information about our API url
base_url = 'http://127.0.0.1:5000'

# Get data about Teams
teams_response = requests.get(base_url + '/teams')
if teams_response.status_code == 200:
    teams_df = pd.read_json(teams_response.text)
else:
    print(f"Something went wrong: {teams_response.text}")

#--------------------------------------------------------------------
# Get list of available seasons from our API
response = requests.get(base_url + '/seasons')
if response.status_code == 200:
    seasons_list = response.json()
else:
    seasons_list = []

# Get list of available conferences from our API
response = requests.get(base_url + '/conferences')
if response.status_code == 200:
    conference_list = response.json()
    conference_list.remove(None)
else:
    conference_list = []
#--------------------------------------------------------------------

app = Dash(__name__)


app.layout = html.Div([
    html.H4('Team Points Per Game'),
    html.P('Season:'),
    dcc.Dropdown(
        id="season_dropdown",
        options=seasons_list,  #  <-- Added list here
        value='2023',
        clearable=False,
    ),
    html.P('Conference:'),
    dcc.Dropdown(
        id="conference_dropdown",
        options=conference_list,  #  <-- Added list here
        value='SEC',
        clearable=False,
    ),
    dcc.Graph(id="graph"),
])


@app.callback(
    Output("graph", "figure"), 
    Input("season_dropdown", "value"), 
    Input("conference_dropdown", "value"))
def update_bar_chart(season, conference):

    # Get data about SEC team's 2023 season PPGs
    ppg_response = requests.get(base_url + f'/ppg?conference={conference}&season={season}')

    if ppg_response.status_code == 200:

        # Plotly uses dataframes, so let's use Pandas to read in the json into a dataframe
        ppg_df = pd.read_json(ppg_response.text)

        # Merge Teams and PPG dataframes using the ID column 
        merged_df = ppg_df.merge(teams_df, left_on='id', right_on='id')
        merged_df.sort_values('ppg') # Make sure our dataframe is still sorted correctly after the merge

        # Now let's create a horizontal bar graph with Plotly and use the team colors to color code our bars
        fig = px.bar(merged_df,
                    x="ppg", 
                    y="team", 
                    orientation='h',
                    hover_data=['rank','id','team','ppg','mascot'],
                    height=400,
                    color='team',
                    color_discrete_sequence=merged_df['color'],
                    title='Team Points Per Game')
        
        # return the graph
        return fig
    else:
        print(f"Something went wrong: {ppg_response.text}")
    

app.run_server(debug=True)

<br/>
<hr/>
<br/>

## User Inputs with Jupyter Notebook Widgets
Even though I'm using a Jupyter Notebook, I'd like to have some easy to use user input widgets for my users to use so that they don't have to try to update my code themselves. **Jupyter Notebook Widgets** are  a great solution for this. 
To use these, you'll need to install the `ipywigets` library. [Here](https://saturncloud.io/blog/understanding-jupyter-notebook-widgets/#:~:text=Widgets%20in%20Jupyter%20Notebook%20are,%2C%20text%20boxes%2C%20and%20more.) is a great tutorial that introduces this library, and [here](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html) is more information on the available widgets, but you can always find a lot more information on Google.

*In your terminal:*
```bash
pip install ipywidgets
```

Let's start out by looking at their example...

In [ ]:
import ipywidgets as widgets

# Create dropdown
my_dropdown = widgets.Dropdown(
                                options=['1', '2', '3'],
                                value='2',
                                description='Number:',
                                disabled=False,
                              )

# Display dropdown
my_dropdown

Here is another way that automatically generate the right type of widgets and allows us to run a function anytime they change. See [this](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html) for some more examples.

In [ ]:
from ipywidgets import interact

@interact
def print_number_dropdown(Number=['1', '2', '3']):
        print(f"The chosen number is {Number}!")


<br/>
Great! Now that that works, let's create a dropdown that uses our API to populate it's options.

In [ ]:
from ipywidgets import interact
import requests

# Get list of available conferences from our API
response = requests.get('http://127.0.0.1:5000/conferences')
if response.status_code == 200:
    conference_list = response.json()
else:
    conference_list = []

# Create conference dropdown and define function to run when it is updated
@interact
def print_conference(Conference=conference_list):
    print(f"The chosen conference is {Conference}!")


Let's change it up now, and use some widgets to manipulate data in our system through the API endpoints.



In [ ]:
import ipywidgets as widgets
import requests
import json

# Create text widgets
id = widgets.Text(description='ID:')
school = widgets.Text(description='School:')
mascot = widgets.Text(description='Mascot:')
abbreviation = widgets.Text(description='Abbreviation:')
conference = widgets.Text(description='Conference:')
color = widgets.Text(description='Color:')
alt_color = widgets.Text(description='Alt Color:')
status = widgets.Text(disabled=True)

# Create button widgets
get_button = widgets.Button(description='GET', button_style='success')
put_button = widgets.Button(description='UPDATE', button_style='warning')
clear_button = widgets.Button(description='Clear All', button_style='info', icon='eraser')

# Group and show widgets
id_box = widgets.HBox([id, status])
button_box = widgets.HBox([get_button, put_button, clear_button])
display(id_box, button_box, school, mascot, abbreviation, conference, color, alt_color)

# Function to clear our fields for us (except ID)
def clear_fields(b=None):
    school.value = ''
    mascot.value = ''
    abbreviation.value = ''
    conference.value = ''
    color.value = ''
    color.style.background = 'white'
    alt_color.value = ''
    alt_color.style.background = 'white'
    if b is not None:
        id.value = ''
        status.value = ''
        status.style.background = 'white'

# functions to interact with the API
def get_team(b):
    response = requests.get(f'http://127.0.0.1:5000/teams/{id.value}')
    if response.status_code == 200:
        team_data = response.json()
        school.value = str(team_data['school'])
        mascot.value = str(team_data['mascot'])
        abbreviation.value = str(team_data['abbreviation'])
        conference.value = str(team_data['conference'])
        color.value = str(team_data['color'])
        if str(team_data['color']) != 'None': color.style.background = team_data['color']
        alt_color.value = str(team_data['alt_color'])
        if str(team_data['alt_color']) != 'None': alt_color.style.background = team_data['alt_color']
        status.value = 'SUCCESS'
        status.style.background = 'lightgreen'
    else:
        clear_fields()
        status.value = f'ERROR - {response.status_code}'
        status.style.background = 'pink'

def put_team(b):
    team_data = json.dumps({
        'school': school.value,
        'mascot': mascot.value,
        'abbreviation': abbreviation.value,
        'conference': conference.value,
        'color': color.value,
        'alt_color': alt_color.value
    })
    headers = {'Content-Type': 'application/json'}
    response = requests.put(f'http://127.0.0.1:5000/teams/{id.value}', headers=headers, data=team_data)
    if response.status_code == 200:
        clear_fields()
        status.value = 'SUCCESS'
        status.style.background = 'lightgreen'
    else:
        status.value = f'ERROR - {response.status_code}'
        status.style.background = 'pink'


# Connect our buttons to our API functions
get_button.on_click(get_team)
put_button.on_click(put_team)
clear_button.on_click(clear_fields)
